# BEN Tutorial

This tutorial copies almost directly from the documentation, it is recommended that
the user use this only for reference or if they just want to validate that the module
is operating properly without having to copy-paste.


Note: You will have to download the relevant files in order to run this.

In [ ]:
from gerrytools.ben import *

### Compression

In [ ]:
ben(
    mode="encode",
    input_file_path="./small_example.jsonl",
)

In [ ]:
ben(
    mode="x-encode",
    input_file_path="./small_example.jsonl.ben",
)

In [ ]:
ben(
    mode="decode",
    input_file_path="./small_example.jsonl.ben",
    output_file_path="./re_small_example.jsonl",
)

In [ ]:
ben(
    mode="x-decode",
    input_file_path="./small_example.jsonl.xben",
    output_file_path="./re_small_example_v2.jsonl",
)

In [ ]:
ben(
    mode="xz-compress",
    input_file_path="./small_example.jsonl",
    output_file_path="./compressed_small_example.jsonl.xz",
)

In [ ]:
ben(
    mode="xz-decompress",
    input_file_path="./compressed_small_example.jsonl.xz",
    output_file_path="./decompressed_small_example.jsonl",
)

### Improving Compression via Relabeling

In [ ]:
ben(
    mode="decode",
    input_file_path="100k_CO_chain.jsonl.xben"
)

In [ ]:
canonicalize_ben_file(
    input_file_path="100k_CO_chain.jsonl.ben"
)

In [ ]:
relabel_json_file_by_key(
    dual_graph_path="CO_small.json",
    key="GEOID20",
    # uncomment the next line if you are running this in a Jupyter Notebook
    verbose=False
)

In [ ]:
relabel_ben_file_with_map(
    input_file_path="100k_CO_chain_canonicalized_assignments.jsonl.ben",
    map_file_path="CO_small_sorted_by_GEOID20_map.json"
)

In [ ]:
ben(
    mode="x-encode",
    input_file_path="100k_CO_chain_canonicalized_assignments_sorted_by_GEOID20.jsonl.ben"
)

### Parsing Forest Recom and SMC Output

In [ ]:
import json

with open("./NC_pct21/42_atlas_gamma0.0_10.jsonl") as f:
    for i, line in enumerate(f):
        if i == 2:
            print(json.loads(line)["levels in graph"])
            break

In [ ]:
msms_parse(
    mode="standard_jsonl",
    region="county",
    subregion="prec_id",
    dual_graph_path="./NC_pct21.json",
    input_file_path="./NC_pct21/42_atlas_gamma0.0_10.jsonl",
    output_file_path="./NC_pct21/42_atlas_gamma0.0_10_standardized.jsonl"
)

In [ ]:
smc_parse(
    mode="standard_jsonl",
    input_file_path="./4x4_grid/SMC_42_29_assignments.csv",
    output_file_path="./4x4_grid/SMC_42_29.jsonl"
)

### Replaying a Chain

In [ ]:
from gerrychain import Graph, Partition
from gerrychain.updaters import Tally

graph = Graph.from_json("CO_small.json")
def pop_tally(graph, new_assignment):
    partition = Partition(
        graph=graph,
        assignment=new_assignment,
        updaters={
            "population": Tally("TOTPOP20", alias="population"),
        }
    )
    return partition["population"]

In [ ]:
for i, assignment in enumerate(ben_replay("100k_CO_chain.jsonl.ben")):
    print(pop_tally(graph, assignment))
    if i > 9:
        break